In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.1.2-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-09 20:41:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-09 20:41:03 (6.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://mohammadassaad-bucket.s3.amazonaws.com/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
from pyspark.sql.functions import to_date
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [14]:
votes_df = vine_df.filter("total_votes>=20")
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [15]:
helpful_votes_df = votes_df.filter("helpful_votes/total_votes>=0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [18]:
vine_yreview_df = helpful_votes_df.filter("vine == 'Y'")
vine_yreview_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|
| RNP01HW9YISJO|          4|           20|         23|   Y|                N|
|R3KLACA6LCDZ0S|          3|           21|         22|   Y|                N|
| RZEQYOT2RE0N7|          4|           75|         80|   Y|     

In [19]:
vine_nreview_df = helpful_votes_df.filter("vine == 'N'")
vine_nreview_df.show() 

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [37]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from collections import Counter
import re
import pyspark.sql.functions as f

In [63]:
reviews = helpful_votes_df.count()
reviews

65581

In [57]:
reviews_vine_no = vine_nreview_df.count()
reviews_vine_no

64968

In [58]:
reviews_vine_yes = vine_yreview_df.count()
reviews_vine_yes

613

In [59]:
unpaid5 = vine_nreview_df.filter("star_rating == 5").count()
unpaid5

30543

In [60]:
paid5 = vine_yreview_df.filter("star_rating == 5").count()
paid5

222

In [61]:
unpaid5_per = unpaid5 / reviews_vine_no *100
unpaid5_per

47.01237532323606

In [62]:
paid5_per = paid5 / reviews_vine_yes *100
paid5_per

36.215334420880914